Este notebook recorre los archivos html de la carpeta `./data/raw`, y exporta de ahi el js del cuatrimestre

Ese `horarios.js` hay que ponerlo en la carpeta src/data

La carpeta data tiene que tener un archivo html por **cada carrera**

El html se consigue de la siguiente manera
- Loggearse al SIU, ir a "Reportes" > "Oferta de comisiones"
- Conseguir el HTML de ese sitio (CTRL+U)

In [53]:
import os
import html
from bs4 import BeautifulSoup

replacements = {
    r'\"': r'"',
    r'<\/': r'</',
    r'\n': '',
    '\n': '',
    '\t': '',
    r'\t': '',
    'á':'a',
    'é':'e',
    'í':'i',
    'ó':'o',
    'ú':'u',
    'Á':'A',
    'É':'E',
    'Í':'I',
    'Ó':'O',
    'Ú':'U'
}

tildes = {
    "\\u00c1": "Á",
    "\\u00e1": "á",
    "\\u00c9": "É",
    "\\u00e9": "é",
    "\\u00cd": "Í",
    "\\u00ed": "í",
    "\\u00d3": "Ó",
    "\\u00f3": "ó",
    "\\u00da": "Ú",
    "\\u00fa": "ú",
    "\\u00dc": "Ü",
    "\\u00fc": "ü",
    "\\u00d1": "Ṅ",
    "\\u00f1": "ñ"
}


for f in sorted(os.listdir('raw'), key=os.path.getmtime):
    with open('raw/'+f, 'r') as fsub:
        print(f, end="")
        txt = fsub.read()
        txt = html.unescape(txt)
        for k,v in replacements.items():
            txt = txt.replace(k,v)
        start = txt.find("kernel.renderer.on_arrival({")
        end = txt.find("})", start)
        content = txt.find('content', start, end)
        contentstart = txt.find('<', content)
        contentend = txt.find('>",', contentstart)
        if contentend == -1: contentend = txt.find(">',", contentstart)
        txt = txt[contentstart+1:contentend+1]
        soup = BeautifulSoup(txt, 'html.parser')
        ultimocuatri = soup.find_all('div', {'class': 'js-recuadro_periodo'})[-1]
        with open('clean/'+f, 'w') as newf:
            newtxt = str(ultimocuatri)
            for k,v in tildes.items():
                newtxt = newtxt.replace(k,v)
            newf.write(newtxt)
        print(" -> clean/" + f)
        
        

petroleo.html -> clean/petroleo.html
informatica.html -> clean/informatica.html
industrial.html -> clean/industrial.html
quimica.html -> clean/quimica.html
sistemas86.html -> clean/sistemas86.html


In [19]:
from collections import namedtuple
import re
import json

REGEX = r'(.*) (\(\d\d\d\d\))'
materia_regex = re.compile(REGEX)
DIAS_DE_LA_HERMOSA_SEMANA = ["Domingo", "Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado"]

In [20]:
def get_ids_materias(soup):
    codigos_materias = []
    actividades = soup.find_all('div', {'class': 'js-recuadro_actividad'})
    for act in actividades:
        nombre, codigo = materia_regex.search(act.get('actividad')).groups()
        codigo = re.sub('\(|\)', "", codigo)
        codigos_materias.append(codigo)
    return codigos_materias

soup = BeautifulSoup(open("clean/informatica.html"), "html.parser")
get_ids_materias(soup)

['7544',
 '7546',
 '6108',
 '7540',
 '7541',
 '7507',
 '6606',
 '7509',
 '6103',
 '6110',
 '7512',
 '7538',
 '7141',
 '7573',
 '6626',
 '7515',
 '7548',
 '6669',
 '6405',
 '7112',
 '6670',
 '7118',
 '6201',
 '6203',
 '6215',
 '6761',
 '7802',
 '7803',
 '7801',
 '7804',
 '7113',
 '7146',
 '7543',
 '7550',
 '6602',
 '7140',
 '7514',
 '7565',
 '6107',
 '7201',
 '6211',
 '7114',
 '7115',
 '7120',
 '6620',
 '7506',
 '7556',
 '6109',
 '6646',
 '6301',
 '7144',
 '6663',
 '6632',
 '7571',
 '7572',
 '6674',
 '7526',
 '7567',
 '7570',
 '6649',
 '6617',
 '7574',
 '6671',
 '7508',
 '7545',
 '7547',
 '7555',
 '7542',
 '7552',
 '7561',
 '6635',
 '7510',
 '7551',
 '7559',
 '7529',
 '7530',
 '6624',
 '7531']

In [21]:
def get_ids_comisiones(soup):
    ids_comisiones = []
    comisiones_soup = soup.find_all('table', {'class': 'comision'})
    for comision in comisiones_soup:
        comision_id = comision.get('comision')
        ids_comisiones.append(comision_id)
    return ids_comisiones

def get_materias(soup):
    materias = []
    actividades = soup.find_all('div', {'class': 'js-recuadro_actividad'})
    for act in actividades:
        nombre, codigo = materia_regex.search(act.get('actividad')).groups()
        codigo = re.sub('\(|\)', "", codigo)
        ids_comisiones = get_ids_comisiones(act)
        
        materias.append({
            "codigo": codigo,
            "nombre": nombre,
            "cursos": ids_comisiones
        })
        
    return materias

soup = BeautifulSoup(open("clean/informatica.html"), "html.parser")
get_materias(soup)

[{'codigo': '7544',
  'nombre': 'ADM. Y CONTROL DE PROY. INFORMATICOS I',
  'cursos': ['31710', '31711']},
 {'codigo': '7546',
  'nombre': 'ADM. Y CONTROL DE PROY. INFORMATICOS II',
  'cursos': ['31712', '31713']},
 {'codigo': '6108',
  'nombre': 'ALGEBRA II A',
  'cursos': ['31746',
   '31747',
   '31748',
   '31749',
   '31750',
   '31751',
   '31752',
   '31753',
   '31754',
   '31755',
   '31756',
   '31757',
   '31758',
   '31759',
   '31760',
   '31761',
   '31762',
   '31763',
   '31764',
   '31765',
   '31766',
   '33815',
   '33816',
   '33817',
   '33818']},
 {'codigo': '7540',
  'nombre': 'ALGORITMOS Y PROGRAMACION I',
  'cursos': ['31773', '31774', '31776', '31777', '31778']},
 {'codigo': '7541',
  'nombre': 'ALGORITMOS Y PROGRAMACION II',
  'cursos': ['31790', '31791', '31792', '31793', '33770']},
 {'codigo': '7507',
  'nombre': 'ALGORITMOS Y PROGRAMACION III',
  'cursos': ['31802', '31804', '33673']},
 {'codigo': '6606',
  'nombre': 'ANALISIS DE CIRCUITOS',
  'cursos': ['

In [22]:
def get_comision(soup, codigo):
    comision_soup = soup.find('table', {'comision': codigo})
    docentes = comision_soup.get('docentes')
    comision_id = comision_soup.get('comision')
    clases = []

    for dia_semana_soup in comision_soup.find_all('tr', {'class': ['js-dia']}):
        dia_semana = dia_semana_soup.get('dia_sem')
        horas_soup = dia_semana_soup.find('td', text=re.compile(r'\d\d\:\d\d'))
        if not horas_soup:
            continue
        horas = horas_soup.contents[0]
        inicio, fin = horas.split(' a ')

        clases.append({'dia': DIAS_DE_LA_HERMOSA_SEMANA.index(dia_semana), 'inicio': inicio, 'fin': fin})

    return {
        "docentes": docentes,
        "codigo": codigo,
        "clases": clases
    }

soup = BeautifulSoup(open("clean/informatica.html"), "html.parser")
get_comision(soup, "31792")

{'docentes': 'BUCHWALD MART\\u00cdN EZEQUIEL, COLLINET JORGE FRANCISCO, GENENDER PE\\u00d1A EZEQUIEL DAVID, GONZALEZ LISELLA FIONA CARLA, SELLA FAENA JASMINA, SIM\\u00d3 PIQUERES ADEODATO',
 'codigo': '31792',
 'clases': [{'dia': 1, 'inicio': '17:30', 'fin': '19:00'},
  {'dia': 1, 'inicio': '19:00', 'fin': '20:30'},
  {'dia': 5, 'inicio': '17:30', 'fin': '19:00'},
  {'dia': 5, 'inicio': '19:00', 'fin': '20:30'}]}

In [ ]:
import datetime

NOMBRES = {
    "clean/informatica.html": "Ingeniería en Informática",
    "clean/civil.html": "Ingeniería Civil",
    "clean/industrial.html": "Ingeniería Industrial",
    "clean/mecanica.html": "Ingeniería Mecánica",
    "clean/quimica.html": "Ingeniería Química",
    "clean/sistemas.html": "Licenciatura en Análisis de Sistemas",
    "clean/sistemas86.html": "Licenciatura en Análisis de Sistemas (Plan viejo)",
    "clean/electronica.html": "Ingeniería Electrónica",
    "clean/alimentos.html": "Ingeniería de Alimentos",
    "clean/agrimensura.html": "Ingeniería en Agrimensura",
    "clean/naval-mecanica.html": "Ingeniería Naval y Mecánica",
    "clean/petroleo.html": "Ingeniería en Petroleo",
    "clean/electricista.html": "Ingeniería Electricista"
}

carreras = []
codigos_materias_agregadas = []
materias = []
codigos_comisiones_agregadas = []
comisiones = []

for f in sorted(os.listdir('clean'), key=os.path.getmtime, reverse=True):
    f = 'clean/'+f
    soup = BeautifulSoup(open(f), "html.parser")

    carreras.append({
        "nombre": NOMBRES[f],
        'materias': get_ids_materias(soup)
    })
    
    materias_carrera = get_materias(soup)
    materias_carrera = [x for x in materias_carrera if x['codigo'] not in codigos_materias_agregadas]
    for m in materias_carrera:
        codigos_materias_agregadas.append(m['codigo'])
        comisiones_materia = m.get('cursos')
        comisiones_materia = [x for x in comisiones_materia if x not in codigos_comisiones_agregadas]
        for c in comisiones_materia:
            codigos_comisiones_agregadas.append(c)
            comisiones.append(get_comision(soup, c))
            
        comisiones_sin_clases = filter(lambda x: not x['clases'], comisiones)
        comisiones_sin_clases_codigos = map(lambda x: x['codigo'], comisiones_sin_clases)
        m['cursos'] = list(filter(lambda x: x not in comisiones_sin_clases_codigos, m['cursos']))
    
    comisiones = list(filter(lambda x: x['clases'], comisiones))
    materias.extend(materias_carrera) 
    
obj = {
    "cuatrimestre": "2021C1",
    "timestamp": str(datetime.datetime.now()),
    "carreras": sorted(carreras, key=lambda x: x['nombre']),
    "materias": sorted(materias, key=lambda x: x['codigo']),
    "cursos": sorted(comisiones, key=lambda x: x['codigo'])
}

obj_dump = json.dumps(obj, indent=2, ensure_ascii=False)        

with open('horarios.js', 'w') as fw:
    fw.write("export const data = ")
    fw.write(obj_dump)

print(obj_dump)    